In [65]:
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from snowflake.snowpark.functions import udf
%matplotlib inline
import datetime as dt
import numpy as np
import seaborn as sns

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import MinMaxScaler
import snowflake.connector
# to save the trained scaler class
import joblib

#Snowflake connection info
# from config import snowflake_conn_prop

import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt



snowflake_conn_prop = {
   "account": "se58322-fsesg",
   "user": "Muhammad Nauman",
   "password": "Muhammad23",
   "role": "RL_ESG_OBJECTS_DEVELOPER",
   "database": "DM_ESG_DEV",
   "schema": "ESG_MODELS",
   "warehouse": "WH_ESG_SUSTAINABILITY",
}

from snowflake.snowpark import version
print(version.VERSION)


connection_paramter = snowflake.connector.connect(account = "se58322-fsesg",
                        user="Muhammad Nauman",
                        password="Muhammad23",
                        role="RL_ESG_OBJECTS_DEVELOPER",
                        warehouse="WH_ESG_SUSTAINABILITY",
                        ocsp_fail_open=False)

session = Session.builder.configs(snowflake_conn_prop).create()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

(1, 0, 0)
[Row(CURRENT_WAREHOUSE()='WH_ESG_SUSTAINABILITY', CURRENT_DATABASE()='DM_ESG_DEV', CURRENT_SCHEMA()=None)]


In [3]:
# 
# qry = "SELECT * FROM DL_ESG_DEV.CUSTOMER.CUSTOMER"
qry = 'SELECT * FROM DL_ESG_DEV.ESG.TRIAL_SCO_ESG_262'

df = pd.read_sql(qry, connection_paramter)

esg_df = df.copy()
# articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
esg_df.head(5)


qry = 'SELECT * FROM DL_ESG_DEV.CUSTOMER.PORTFOLIO'

df_port = pd.read_sql(qry, connection_paramter)

port_df = df_port.copy()
port_df.shape


/var/folders/nr/vyfq21gx5pn1p0025dclx97c0000gn/T/ipykernel_43165/213873629.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry, connection_paramter)
/var/folders/nr/vyfq21gx5pn1p0025dclx97c0000gn/T/ipykernel_43165/213873629.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_port = pd.read_sql(qry, connection_paramter)


(40000, 9)

In [4]:
#############################
#Recommendation System Based on Movie Overviews
#############################
# 1. Creating the TF-IDF Matrix
# 2. Creating the Cosine Similarity Matrix
# 3. Making Suggestions Based on Similarities
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# df = pd.read_csv("movies_metadata.csv",
#                  low_memory=False) # To turn off DtypeWarning
# #https://drive.google.com/file/d/1M1ozB3kGCoDCGLka8clMXz0lX9nvo42X/view , if you want to access the data.
# df.shape
# OUT  = (45466, 24)   #24 Columns , 45466 row
# df.head()

In [5]:
#Important colomn for us is 'overview' .
esg_df.head()


esg_df['summary'] = df.apply(lambda row:   row['exch_region'] + " " + row['economic_sector'] + " " + row['industry'], axis=1)


In [6]:
tfidf = TfidfVectorizer(stop_words='english') 
#stop_words english is for Ignoring 'The' ,'a' etc. 
esg_df['summary'] = esg_df['summary'].fillna('')
tfidf_matrix = tfidf.fit_transform(esg_df['summary'])
tfidf_matrix.shape
# OUT = (45466, 75827) 

(495, 169)

In [7]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
#Since these two matrices are similar to each other, we wrote both of them at the same time.
cosine_sim.shape

(495, 495)

In [8]:
esg_df


,date,asset_id,name,ticker,exchange_iso,exchange_mic,dom_country_iso,dom_region,exch_country_iso,exch_region,economic_sector,industry,esg,esg_e,esg_s,esg_g,summary
0,2018-01-01,196,A. O. Smith Corporation,AOS,NYS,XNYS,USA,North America,USA,North America,Producer Manufacturing,Building Products,55.67,46.93,56.43,62.57,North America Producer Manufacturing Building ...
1,2018-01-01,208,Abbott Laboratories,ABT,NYS,XNYS,USA,North America,USA,North America,Health Technology,Medical Specialties,46.78,68.29,72.38,17.89,North America Health Technology Medical Specia...
2,2018-01-01,209,"AbbVie, Inc.",ABBV,NYS,XNYS,USA,North America,USA,North America,Health Technology,Pharmaceuticals: Major,51.41,65.98,62.65,35.37,North America Health Technology Pharmaceutical...
3,2018-01-01,270,Adobe Inc.,ADBE,NAS,XNAS,USA,North America,USA,North America,Technology Services,Packaged Software,64.47,68.40,67.56,61.60,North America Technology Services Packaged Sof...
4,2018-01-01,345,"Akamai Technologies, Inc.",AKAM,NAS,XNAS,USA,North America,USA,North America,Technology Services,Internet Software/Services,57.44,73.85,62.76,49.76,North America Technology Services Internet Sof...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,2018-01-01,94362,AT&T Inc.,T,SWX,XSWX,USA,North America,CHE,Europe,Communications,Major Telecommunications,53.34,63.41,51.20,49.93,Europe Communications Major Telecommunications
491,2018-01-01,94615,Linde plc,LIN,ETR,XETR,GBR,Europe,DEU,Europe,Process Industries,Chemicals: Specialty,64.71,70.91,73.87,56.78,Europe Process Industries Chemicals: Specialty
492,2018-01-01,94676,International Flavors & Fragrances Inc.,IFF,TAE,XTAE,USA,North America,ISR,Middle East,Consumer Non-Durables,Household/Personal Care,60.64,68.93,70.07,49.53,Middle East Consumer Non-Durables Household/Pe...
493,2018-01-01,94720,Exxon Mobil Corporation,XONA,ETR,XETR,USA,North America,DEU,Europe,Energy Minerals,Integrated Oil,55.49,58.74,54.11,54.64,Europe Energy Minerals Integrated Oil


In [9]:
esg_df = esg_df[~esg_df["ticker"].isna()]
#Deletion of titles with Nan Values
indices = pd.Series(esg_df.index, index=esg_df['ticker'])
#Let's catch the indexes of all the movies.
indices.head()

ticker
AOS     0
ABT     1
ABBV    2
ADBE    3
AKAM    4
dtype: int64

In [10]:
#Delete to duplicated ones.
indices = indices[~indices.index.duplicated(keep='last')]
indices.shape

(494,)

In [11]:

movie_index = indices["AOS"]
# movie_index = 35116 
cosine_sim[movie_index]

similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])
similarity_scores.sort_values(by='score',ascending=False).head(10)

,score
0,1.000000
336,0.939925
282,0.939925
296,0.939925
111,0.654808
77,0.654808
29,0.654808
246,0.581931
330,0.581931
208,0.455760


In [12]:
# movie_indices
# esg_df.iloc[movie_indices]

NameError: name 'movie_indices' is not defined

In [22]:
ticker = 'ROK'


movie_index = indices[ticker]
# movie_index = 35116 
cosine_sim[movie_index]

similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])
similarity_scores.sort_values(by='score',ascending=False).head(10)

movie_indices = similarity_scores.sort_values("score", ascending=False)[0:11].index


esg_df.iloc[movie_indices].head(10)
# esg_df[movie_indices]

,date,asset_id,name,ticker,exchange_iso,exchange_mic,dom_country_iso,dom_region,exch_country_iso,exch_region,economic_sector,industry,esg,esg_e,esg_s,esg_g,summary
111,2018-01-01,7403,Generac Holdings Inc.,GNRC,NYS,XNYS,USA,North America,USA,North America,Producer Manufacturing,Electrical Products,45.95,36.42,48.68,50.67,North America Producer Manufacturing Electrica...
77,2018-01-01,4710,"Rockwell Automation, Inc.",ROK,NYS,XNYS,USA,North America,USA,North America,Producer Manufacturing,Electrical Products,62.77,55.94,60.88,69.34,North America Producer Manufacturing Electrica...
29,2018-01-01,1817,Eaton Corp. Plc,ETN,NYS,XNYS,IRL,Europe,USA,North America,Producer Manufacturing,Electrical Products,65.65,79.31,67.94,54.53,North America Producer Manufacturing Electrica...
247,2018-01-01,92224,"Mondelez International, Inc. Class A",0R0G,LON,XLON,USA,North America,GBR,Europe,Consumer Non-Durables,Food: Major Diversified,56.77,58.60,64.14,50.88,Europe Consumer Non-Durables Food: Major Diver...
331,2018-01-01,92600,"Cadence Design Systems, Inc.",0HS2,LON,XLON,USA,North America,GBR,Europe,Technology Services,Packaged Software,57.09,56.76,59.51,56.03,Europe Technology Services Packaged Software
0,2018-01-01,196,A. O. Smith Corporation,AOS,NYS,XNYS,USA,North America,USA,North America,Producer Manufacturing,Building Products,55.67,46.93,56.43,62.57,North America Producer Manufacturing Building ...
337,2018-01-01,92620,Capital One Financial Corporation,0HT4,LON,XLON,USA,North America,GBR,Europe,Finance,Major Banks,54.58,69.45,59.26,44.65,Europe Finance Major Banks
283,2018-01-01,92384,"F5, Inc.",0IL6,LON,XLON,USA,North America,GBR,Europe,Electronic Technology,Computer Communications,65.86,49.59,61.95,75.67,Europe Electronic Technology Computer Communic...
297,2018-01-01,92436,Mastercard Incorporated Class A,0R2Z,LON,XLON,USA,North America,GBR,Europe,Finance,Finance/Rental/Leasing,59.03,59.11,52.28,62.49,Europe Finance Finance/Rental/Leasing
209,2018-01-01,90585,"Cisco Systems, Inc.",4333,HKG,XHKG,USA,North America,HKG,Asia,Technology Services,Information Technology Services,61.85,79.12,67.12,50.78,Asia Technology Services Information Technolog...


In [29]:
for a, b in esg_df[['ticker', 'name']].itertuples(index=False):
    print(a, b)

AOS A. O. Smith Corporation
ABT Abbott Laboratories
ABBV AbbVie, Inc.
ADBE Adobe Inc.
AKAM Akamai Technologies, Inc.
ALGN Align Technology, Inc.
ABC AmerisourceBergen Corporation
ADI Analog Devices, Inc.
ADM Archer-Daniels-Midland Company
ADSK Autodesk, Inc.
BIIB Biogen Inc.
BIO Bio-Rad Laboratories, Inc. Class A
CHRW C.H. Robinson Worldwide, Inc.
CPB Campbell Soup Company
CAT Caterpillar Inc.
CRL Charles River Laboratories International, Inc.
CTAS Cintas Corporation
CTXS Citrix Systems, Inc.
CL Colgate-Palmolive Company
COP ConocoPhillips
CPRT Copart, Inc.
COST Costco Wholesale Corporation
CSX CSX Corporation
DHI D.R. Horton, Inc.
XRAY DENTSPLY SIRONA, Inc.
DXCM DexCom, Inc.
FANG Diamondback Energy, Inc.
DLTR Dollar Tree, Inc.
DOV Dover Corporation
ETN Eaton Corp. Plc
EBAY eBay Inc.
EW Edwards Lifesciences Corporation
EA Electronic Arts Inc.
EQR Equity Residential
EXPD Expeditors International of Washington, Inc.
FAST Fastenal Company
FISV Fiserv, Inc.
FLT FLEETCOR Technologies, Inc.


In [42]:

l_tickers = []
l_companies = []
l_recommendations = []

for ticker, company_name in esg_df[['ticker', 'name']].itertuples(index=False):
    
    movie_index = indices[ticker]
    # movie_index = 35116 
    cosine_sim[movie_index]

    similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])
    similarity_scores.sort_values(by='score',ascending=False).head(10)

    movie_indices = similarity_scores.sort_values("score", ascending=False)[0:11].index
    
    b = np.array([176])
    movie_indices = np.setdiff1d(movie_indices,b)

    # print(movie_indices)
    try:
        mini_df = list(esg_df.loc[movie_indices].head(10)['ticker'].to_numpy())

        l_tickers.append(ticker)
        l_companies.append(company_name)
        l_recommendations.append(mini_df)
    except Exception as e:
        print('skipped', str(e))
    
#     print(list(mini_df['ticker'].to_numpy()))
#     break

# Calling DataFrame constructor after zipping
# both lists, with columns specified
rec_df = pd.DataFrame(list(zip(l_tickers, l_companies, l_recommendations)),
               columns =['Ticker', 'Company', 'Recommendations'])
rec_df

,Ticker,Company,Recommendations
0,AOS,A. O. Smith Corporation,"[AOS, ETN, ITW, ROK, GNRC, IR, 0HF7, 0JZ1, 0IR..."
1,ABT,Abbott Laboratories,"[ABT, ALGN, BIO, XRAY, DXCM, EW, MDT, MTD, TFX..."
2,ABBV,"AbbVie, Inc.","[ABBV, VTRS, KEY, PNC, CTLT, LLY, JNJ, PFE, MR..."
3,ADBE,Adobe Inc.,"[ADBE, ADSK, CTXS, INTU, ORCL, PTC, NLOK, SNPS..."
4,AKAM,"Akamai Technologies, Inc.","[AKAM, ADSK, ORCL, VRSN, PAYC, GOOG, FB2A, TWR..."
...,...,...,...
489,T,AT&T Inc.,"[QCOM, ANET, GEY, 0Q1S, APC, 0HVP, 0IL6, 0KV3,..."
490,LIN,Linde plc,"[DD, 0HC7, 0HQU, 0K3B, 0IFA, 0EDD, 0HBH, 0HUR,..."
491,IFF,International Flavors & Fragrances Inc.,"[CL, KO, PG, 0JTM, 0QOS, 0KEQ, 0I0J, 0K34, 0R1..."
492,XONA,Exxon Mobil Corporation,"[COP, FANG, CVX, 0KAK, 0JY9, 0I8W, 0HGC, 0IDR,..."


In [64]:
# session.create_dataframe(esg_df)

database = 'DM_ESG_DEV'
schema = 'ESG_MODELS'
view_name = 'recommender'

## Code for converting and then saving
snowpark_rec_df = session.write_pandas(rec_df, 'recommender', database=database, schema=schema,  auto_create_table=True, overwrite=True)



In [60]:
snowpark_rec_df.__dir__()

['_session',
 '_plan',
 '_select_statement',
 'is_cached',
 '_reader',
 '_writer',
 '_stat',
 'approxQuantile',
 'approx_quantile',
 'corr',
 'cov',
 'crosstab',
 'sampleBy',
 'sample_by',
 '_na',
 'dropna',
 'fillna',
 'replace',
 'table_name',
 '__module__',
 '__doc__',
 '__init__',
 '__copy__',
 '__enter__',
 '__exit__',
 'sample',
 'update',
 'delete',
 'merge',
 'drop_table',
 'stat',
 'collect',
 'collect_nowait',
 '_internal_collect_with_tag_no_telemetry',
 '_internal_collect_with_tag',
 '_execute_and_get_query_id',
 'to_local_iterator',
 'pandas',
 'to_pandas',
 'to_pandas_batches',
 'to_df',
 '__getitem__',
 '__getattr__',
 'columns',
 'col',
 'select',
 'select_expr',
 'selectExpr',
 'drop',
 'filter',
 'sort',
 'agg',
 'rollup',
 'group_by',
 'group_by_grouping_sets',
 'cube',
 'distinct',
 'drop_duplicates',
 'pivot',
 'unpivot',
 'limit',
 'union',
 'union_all',
 'union_by_name',
 'union_all_by_name',
 '_union_by_name_internal',
 'intersect',
 'except_',
 'natural_join',
 

In [61]:
snowpark_rec_df.table_name

'"recommender"'

In [51]:

database = 'DM_ESG_DEV'
schema = 'ESG_MODELS'
view_name = 'recommender'



snowpark_rec_df.create_or_replace_view(f"{database}.{schema}.{view_name}")

# snowpark_rec_df.show()

[Row(status='View RECOMMENDER successfully created.')]